In [1]:
# ======================================================================
# == load the libraries ==
# ======================================================================

from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label
from stardist.models import StarDist2D
from shapely.geometry import Polygon, Point
import matplotlib.image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import pandas as pd
import numpy as np
import seaborn as sns
import PIL
import tifffile
import sys
import pickle


2025-04-18 16:42:48.999444: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-18 16:42:49.028764: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# ======================================================================
# == definition of custom functions ==
# ======================================================================

from custom_functions import custom_normalize

In [5]:
# ======================================================================
# == processing ==
# ======================================================================

# --- read in the input image ---

# img hires
PIL.Image.MAX_IMAGE_PIXELS = 1718032108
img = tifffile.imread("../../../../data/Mouse_Embryo/Visium_HD_Mouse_Embryo_tissue_image.btf")
img = img[:,:,0:3]

# image type
hist_img = "H&E"

# stardist parameters
# prob_thresh
# nms_thresh

In [6]:
# --- run custom normalization ---

normalized_img = custom_normalize(img)


In [7]:
# --- select approprote stardist model ---

if hist_img == "H&E":
    model_stardist = "2D_versatile_he"
elif hist_img == "IF":
    model_stardist = "2D_versatile_fluo"

model = StarDist2D.from_pretrained(model_stardist)

Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.


In [ ]:
# --- run the model ---

# here is running with default parameters for prob_thresh and nms_thresh
labels, polys = model.predict_instances_big(
    normalized_img, axes='YXC', block_size=4096, min_overlap=128, context=128,
    normalizer=None, # n_tiles=(4,4,1),
    prob_thresh = .05)

In [ ]:
# ======================================================================
# == save the output ==
# ======================================================================

with open("../../../../data/Mouse_Embryo/nuclei_polys.pkl", "wb") as f:  # open a text file
    pickle.dump(polys, f)